In [1]:
import qstrader

In [2]:
import os

import pandas as pd
import pytz

# alpha signal
from qstrader.alpha_model.fixed_signals import FixedSignalsAlphaModel

# asset types and universe
from qstrader.asset.equity import Equity
from qstrader.asset.universe.static import StaticUniverse

# data reader and provider
from qstrader.data.backtest_data_handler import BacktestDataHandler
from qstrader.data.daily_bar_csv import CSVDailyBarDataSource

# stats generator
from qstrader.statistics.tearsheet import TearsheetStatistics

# main thing
from qstrader.trading.backtest import BacktestTradingSession

In [3]:
start_dt = pd.Timestamp('2003-09-30 09:30:00', tz='America/New_York')
end_dt = pd.Timestamp('2019-12-31 16:00:00', tz='America/New_York')

# Construct the symbols and assets necessary for the backtest
strategy_symbols = ['SPY', 'AGG']
strategy_assets = ['EQ:%s' % symbol for symbol in strategy_symbols]
strategy_universe = StaticUniverse(strategy_assets)

In [4]:
os.environ["QSTRADER_CSV_DATA_DIR"] = '/Users/snk/pcode/archive/qstrader/data'

In [5]:
# To avoid loading all CSV files in the directory, set the
# data source to load only those provided symbols
csv_dir = os.environ.get('QSTRADER_CSV_DATA_DIR', '.')
data_source = CSVDailyBarDataSource(csv_dir , Equity, csv_symbols=strategy_symbols)
data_handler = BacktestDataHandler(strategy_universe, data_sources=[data_source])

Loading CSV files into DataFrames...
Loading CSV file for symbol 'EQ:SPY'...
Loading CSV file for symbol 'EQ:AGG'...
Loading pricing in CSV files...
Loading CSV file for symbol 'EQ:SPY'...
Loading CSV file for symbol 'EQ:AGG'...


In [6]:
# Construct an Alpha Model that simply provides
# static allocations to a universe of assets
# In this case 60% SPY ETF, 40% AGG ETF,
# rebalanced at the end of each month
strategy_alpha_model = FixedSignalsAlphaModel({'EQ:SPY': 0.6, 'EQ:AGG': 0.4})
strategy_backtest = BacktestTradingSession(
    start_dt,
    end_dt,
    strategy_universe,
    strategy_alpha_model,
    rebalance='end_of_month',
    long_only=True,
    cash_buffer_percentage=0.01,
    data_handler=data_handler
)
strategy_backtest.run(results = False)

Initialising simulated broker "Backtest Simulated Broker Account"...
(2003-09-30 09:30:00-04:00) - portfolio creation: Portfolio "000001" created at broker "Backtest Simulated Broker Account"
(2003-09-30 09:30:00-04:00) - subscription: 1000000.00 subscribed to portfolio "000001"
Beginning backtest simulation...
(2003-09-30 09:30:00-04:00) - market_open
(2003-09-30 16:00:00-04:00) - market_close
(2003-09-30 16:00:00-04:00) - trading logic and rebalance
(2003-09-30 16:00:00-04:00) - target weights: {'EQ:AGG': 0.4, 'EQ:SPY': 0.6}
(2003-09-30 16:00:00-04:00) - submitted order: EQ:AGG, qty: 3855
(2003-09-30 16:00:00-04:00) - submitted order: EQ:SPY, qty: 5942
(2003-10-01 09:30:00-04:00) - market_open
(2003-10-01 09:30:00-04:00) - executed order: EQ:AGG, qty: 3855, price: 102.64, consideration: 395677.00, commission: 0.00, total: 395677.00
(2003-10-01 09:30:00-04:00) - executed order: EQ:SPY, qty: 5942, price: 100.24, consideration: 595626.00, commission: 0.00, total: 595626.00
(2003-10-01 1

In [ ]:
list(strategy_backtest.stats.keys())

In [ ]:
stats = strategy_backtest.get_stats_dataframe()

In [ ]:
stats['portfolio_snapshots']

In [ ]:
# Construct benchmark assets (buy & hold SPY)
benchmark_assets = ['EQ:SPY']
benchmark_universe = StaticUniverse(benchmark_assets)

# Construct a benchmark Alpha Model that provides
# 100% static allocation to the SPY ETF, with no rebalance
benchmark_alpha_model = FixedSignalsAlphaModel({'EQ:SPY': 1.0})
benchmark_backtest = BacktestTradingSession(
    start_dt,
    end_dt,
    benchmark_universe,
    benchmark_alpha_model,
    rebalance='buy_and_hold',
    long_only=True,
    cash_buffer_percentage=0.01,
    data_handler=data_handler
)
benchmark_backtest.run()

# Performance Output
tearsheet = TearsheetStatistics(
    strategy_equity=strategy_backtest.get_equity_curve(),
    benchmark_equity=benchmark_backtest.get_equity_curve(),
    title='60/40 US Equities/Bonds'
)
tearsheet.plot_results()